# TM10007 Assignment template

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [30]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# fetch data
from worclipo.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(type(data))


The number of samples: 115
The number of columns: 494
<class 'pandas.core.frame.DataFrame'>


# setting up the data to be processed

In [31]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
# change lipoma = 0 and liposarcoma = 1 and encode labels
group_names = list(set(data.label))
data.loc[data['label'] == 'lipoma', 'label'] = 0
data.loc[data['label'] == 'liposarcoma', 'label'] = 1
data['label'] = pd.cut(data['label'], bins = 2, labels=group_names)
print(data['label'].unique())
label_diag = LabelEncoder()
data['label'] = label_diag.fit_transform(data['label'])

# assign X to measurements and y to outcome (lipoma/sarcoma)
X = data.drop('label', axis=1)
y = data['label']

# splitting the data into test and train
from sklearn.model_selection import train_test_split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=10)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=10)

# scale the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# preprocess the data (trainset eventually)
if not str in data.values:
    print(f'All values are numerical.')
if np.isnan(X_train).any():
    imp = SimpleImputer(missing_value=np.nan, strategy='mean')
    X_train.iloc[:,:] = imp.fit_transform(X_train)
    X_test.iloc[:,:] = imp.fit(X_test)
    print(f'All NaN values replaced by mean value of the column.')

# remove features with zero variance
print(f'Not removed: ',X_train.shape)
vt = VarianceThreshold(threshold=0)
X_train = vt.fit_transform(X_train)
X_test = vt.transform(X_test)
print(f'Removed zero variance: ', X_train.shape)


['liposarcoma', 'lipoma']
Categories (2, object): ['lipoma' < 'liposarcoma']
All values are numerical.
Not removed:  (64, 493)
Removed zero variance:  (64, 474)


# Classify

In [32]:

# try a classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

print('rfc', classification_report(y_test, pred_rfc))
print(confusion_matrix(y_test, pred_rfc))

rfc               precision    recall  f1-score   support

           0       0.67      0.62      0.64        13
           1       0.55      0.60      0.57        10

    accuracy                           0.61        23
   macro avg       0.61      0.61      0.61        23
weighted avg       0.61      0.61      0.61        23

[[8 5]
 [4 6]]
